In [ ]:
import time
import requests
import json

import os
from multiprocessing import Process, current_process
import multiprocessing as mp

user_ids = [file_name.split('.')[0] for file_name in os.listdir('saved_data')]

In [ ]:
%%time

def vk_api(method, data, access_token='f54fe3b72906d9d9194ad529c1cd65d8f20e0bee7086f93a8d01ebea4ca66940af1eca4bcf21887f02a66', old_time=time.time()):

    version = '5.37'

    link = "https://api.vk.com/method/{}?access_token={}&v={}".format(method, access_token, version)
    for key, val in data.items():
        link += '&{}={}'.format(key, str(val))

    res = requests.get(link)

    time_delta = time.time() - old_time
    if time_delta < 0.333:
        time.sleep(0.333 - time_delta)
    return res.json(), time.time()

data = {'group_id': 'sci',
        'offset': 0,
        'count': 1000}

old_time = time.time()
for i in range(20):
    try:
        res, old_time = vk_api('groups.getMembers', data, old_time=old_time)
        res = res['response']['items']
    except Exception as e:
        print(str(e))
        print(i)
        print(res)
        break

In [ ]:
users = requests.get('http://78.47.176.17:14288/api/get_all_users').json()

for user in users:
    user['old_time'] = time.time()

users

In [ ]:
vk_api('groups.getById', {'group_id': '163528123', 'fields': 'members_count'})[0]['response'][0]['members_count']

In [ ]:
%%time

 
def get_users(token, user):
    data = {'group_id': 'sci',
            'offset': 0,
            'count': 1000}

    try:
        old_time = user['old_time']
        res, old_time = vk_api('groups.getMembers', data, token, old_time)
        user['old_time'] = old_time

        with open('multiproc.txt') as f:
            old_data = f.read()
        new_data = old_data + '\n' + str(old_time)
        with open('multiproc.txt', 'w') as f:
            f.write(new_data)

        res = res['response']['items']
        res = 'success'
    except Exception as e:
        pass
    proc_name = current_process().name
    print('res: {}\nproc_name: {}\n'.format(res, proc_name))
 

if __name__ == '__main__':
    for user in users:
        user['old_time'] = time.time()

    procs = []
    
    for i in range(20 // len(users)):
        for user in users:
            token = user['token']
            proc = Process(target=get_users, args=(token, user,))
            procs.append(proc)
            proc.start()
        
        for proc in procs:
            proc.join()

In [ ]:
%%time


class ParallelWorker:
    def __init__(self):
        self.break_var = False

    def get_users(self, user, args, return_dict):
        data = {'group_id': args['group_id'],
                'count': 1000,
                'offset': args['offset']}
        
        try:
            old_time = user['old_time']
            token = user['token']
            res, old_time = vk_api('groups.getMembers', data, token, old_time)
            user['old_time'] = old_time
            
            res = res['response']['items']


            proc_name = current_process().name
            with open('multiproc.txt') as f:
                old_data = f.read()
            new_data = old_data + '\n' + str(proc_name) + ' => ' + str(res)
            with open('multiproc.txt', 'w') as f:
                f.write(new_data)

            
            return_dict[proc_name] = res
        except Exception as e:
            print('error!\n', str(e))
            pass
        
    
    
    def parallel_def(self, fn, args=[]):
        users = requests.get('http://78.47.176.17:14288/api/get_all_users').json()

        for user in users: # брать и записывать в бд
            user['old_time'] = time.time()

        procs = []
        manager = mp.Manager()
        return_dict = manager.dict()
        self.break_var = False
        while len(args['var']) > 0:
            for user in users:
                if len(args) == 0:
                    break
                arg = args['var'].pop()
                arg.update(args['static'])
                proc = Process(target=fn, args=(user, arg, return_dict,))
                procs.append(proc)
                proc.start()
            
            for proc in procs:
                proc.join()

        return [val for arr in return_dict.values() for val in arr]

parallel_worker = ParallelWorker()
getted_users = parallel_worker.parallel_def(parallel_worker.get_users, args={'var': [{'offset': i} for i in range(0, 10000, 1000)], 'static': {'group_id': '163528123'}})

In [ ]:
items = Session.query.filter_by(status='in_queue').first()
json.loads(json.dumps(items, cls=AlchemyEncoder))


In [ ]:
[i for i in range(0, 80, 1000)]

In [ ]:
len(list(set(getted_users)))

In [ ]:
a = [1,2,3,4,5]
a[2:4]

In [ ]:
len(list(set(getted_users)))

In [ ]:
(*[i for i in range(5)])

In [ ]:
items = Session.query.filter(not_(Session.status.contains('in_queue'))).all()
json.loads(json.dumps(items, cls=AlchemyEncoder))


In [ ]:
items = Session.query.filter(and_(not_(Session.status.contains('in_queue')), not_(Session.status.contains('complete')))).first()
json.loads(json.dumps(items, cls=AlchemyEncoder))


In [ ]:
session = Session(status='success', data=['1111', '2222'])

try:
    db.session.add(session)
    db.session.commit()
    db.session.rollback()
except Exception as e:
    print(str(e))

In [ ]:
from sqlalchemy import or_, not_, and_

#filter(or_(*[Сertificate.id.like(cert_id) for cert_id in filtred_cert_ids]))